In [1]:
from utils.alloys import vectorization_method_2
from utils.constants import alloy_max_len
from utils.ml import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

2023-10-27 14:19:21.584140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 14:19:22.982233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-27 14:19:26.139806: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2023-10-27 14:19:26.140558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

In [3]:
df = pd.read_csv("final_df.csv")
X = []
for alloy in df["bmg_alloy"]:
    X.append(vectorization_method_2(alloy))
y = df["actual_d_max"]

In [4]:
X = np.array(X, dtype=np.int32)
y = np.array(y, dtype=np.int32)

# have a look at if the vectorization was done correctly - everything was as expected
print(df["bmg_alloy"][0])
print(X[0])
print(y[0])

Ti7.0Cu60.0Zr33.0
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7  0
  0  0  0  0  0 60  0  0  0  0  0  0  0  0  0  0 33  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
3


In [5]:
INPUT_LENGTH = 118
EPOCHS = 4000

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1269, 118) (318, 118) (1269,) (318,)


In [7]:
def test(model, epochs, layer_name=None, test_all=True):
    tf.random.set_seed(42)
    model.compile(optimizer="adam", loss="mean_squared_error")
    global X_train, X_test, y_train, y_test
    history = model.fit(X_train, y_train, verbose=0, epochs=epochs, batch_size=64, validation_data=(X_test, y_test))
    plt.plot(history.history["loss"])
    plt.plot(history.history['val_loss'])
    y_hat = model.predict(X_test)
    print(f"Sequential Model Score: {r2_score(y_test, y_hat)}")
    plt.figure()
    sns.regplot(x=y_test, y=y_hat)

    if (test_all == True):
        selected_layer = model.get_layer(layer_name)
        model_till_selected_layer = tf.keras.Model(inputs=model.input, outputs=selected_layer.output)
        X_train_2 = model_till_selected_layer.predict(X_train)
        X_test_2 = model_till_selected_layer.predict(X_test)
        X_train_2 = pd.DataFrame(data=X_train_2)
        X_test_2 = pd.DataFrame(data=X_test_2)
        y_train_2 = pd.DataFrame(data=y_train)
        y_test_2 = pd.DataFrame(data=y_test)

        display(X_train_2)
        display(X_test_2)
        
        scores = run_all_regressors_with_transformers(X_train=X_train_2, X_test=X_test_2, y_train=y_train_2, y_test=y_test_2, y_transform=True)
        display(scores[:10])

In [8]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(INPUT_LENGTH,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])
print(model_1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                7616      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________
None


2023-10-27 14:24:39.833478: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-27 14:24:39.833777: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (laptop): /proc/driver/nvidia/version does not exist
2023-10-27 14:24:39.835693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
test(model_1, EPOCHS, "dense_1")


KeyboardInterrupt

